In [1]:
#import packages
import requests
import json
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/roadevents?api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [3]:
#summarize the data by park
park_list = []
for x in df["features"]:
    for y in df["road_event_feed_info"]['data_sources']:
        if y['data_source_id']==x['properties']['core_details']['data_source_id']:
            park_list.append(y['organization_name'])

park_df = Counter(park_list)
park_df = pd.DataFrame.from_dict(park_df, orient="index").reset_index().rename(
    columns={'index': 'park',
            0:'number of alerts'}).sort_values(by=['number of alerts'])


In [4]:
# Get today's date as a string
today = datetime.date.today()
today_string = today.strftime('%Y-%m-%d')

park_df['date'] = today_string

In [5]:
len(park_df), sum(park_df['number of alerts']), max(park_df['number of alerts'])

(17, 95, 36)

In [6]:
# Loop through the API entries
counter=0

park_list = []
alert_name_list = []
description_list = []
event_type_list = []
road_name_list = []
vehicle_impact_list = []
start_date_list = []
end_date_list = []
today_date_list = []
id_list = []


for x in df["features"]:
    for y in df["road_event_feed_info"]['data_sources']:
        if y['data_source_id']==x['properties']['core_details']['data_source_id']:
            print("Park:", y['organization_name'])
            park_list.append(y['organization_name'])
    print("Alert name: ", x['properties']['core_details']['name'])
    print("Description: ", x['properties']['core_details']['description'])
    print("Event type: ", x['properties']['core_details']['event_type'])
    print("Road name: ", x['properties']['core_details']['road_names'])
    print("Vehicle impact: ", x['properties']['vehicle_impact'])
    print("Alert ID: ", x['properties']['core_details']['data_source_id'])
  #  print("Dates: ", x['properties']['start_date'], "to ", x['properties']['end_date'])
    print("")
    alert_name_list.append(x['properties']['core_details']['name'])
    description_list.append(x['properties']['core_details']['description'])
    event_type_list.append(x['properties']['core_details']['event_type'])
    road_name_list.append(x['properties']['core_details']['road_names'])
    vehicle_impact_list.append(x['properties']['vehicle_impact'])
    start_date_list.append(x['properties']['start_date'])
    id_list.append(x['properties']['core_details']['data_source_id'])
   # end_date_list.append(x['properties']['end_date'])
    today_date_list.append(today_string)
    counter+=1

Park: Crater Lake National Park
Alert name:  Road from Park Headquarters to Rim Village is Temporarily CLOSED
Description:  The road from Park Headquarters to Rim Village is temporarily closed due to the latest snowstorm.
Event type:  incident
Road name:  ['Munson Valley Road from Park Headquarters to Rim Village']
Vehicle impact:  all-lanes-closed
Alert ID:  02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2

Park: Crater Lake National Park
Alert name:  Road from Park Headquarters to Rim Village is Temporarily CLOSED
Description:  The road from Park Headquarters to Rim Village is temporarily closed due to the latest snowstorm.
Event type:  incident
Road name:  ['Munson Valley Road from Park Headquarters to Rim Village']
Vehicle impact:  all-lanes-closed
Alert ID:  02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2

Park: Crater Lake National Park
Alert name:  Seasonal Road Closure
Description:  North Entrance Road, West Rim Drive, and East Rim Drive are CLOSED for the season.  
Event type:  incident
Road name:  

In [7]:
df_update = pd.DataFrame({
    "date":today_date_list,
    "park":park_list,
    "alert name": alert_name_list,
    "description": description_list,
    "event type": event_type_list,
    "road name": road_name_list,
    "vehicle impact": vehicle_impact_list,
    "event start date": start_date_list,
    "alert_id": id_list
#    "event end date": end_date_list
})
df_update.head

<bound method NDFrame.head of           date                                        park  \
0   2024-03-28                   Crater Lake National Park   
1   2024-03-28                   Crater Lake National Park   
2   2024-03-28                   Crater Lake National Park   
3   2024-03-28                   Crater Lake National Park   
4   2024-03-28                   Crater Lake National Park   
5   2024-03-28                   Crater Lake National Park   
6   2024-03-28                   Crater Lake National Park   
7   2024-03-28                   Crater Lake National Park   
8   2024-03-28                   Crater Lake National Park   
9   2024-03-28                   Crater Lake National Park   
10  2024-03-28                   Crater Lake National Park   
11  2024-03-28                   Crater Lake National Park   
12  2024-03-28                   Crater Lake National Park   
13  2024-03-28                   Crater Lake National Park   
14  2024-03-28                    Shenan

In [8]:
path = "Park Unit Scraping Information.csv"
regionlookup = pd.read_csv(path, encoding='latin-1')
regionlookup.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Abraham Lincoln Birthplace,Abraham Lincoln Birthplace National Historical...,National Historical Park,KY,SER,ABLI,1.0
1,Acadia,Acadia National Park,National Park,ME,NER,ACAD,1.0
2,Adams,Adams National Historical Park,National Historical Park,MA,NER,ADAM,1.0
3,African Burial Ground,African Burial Ground National Monument,National Monument,NY,NER,AFBG,1.0
4,Agate Fossil Beds,Agate Fossil Beds National Monument,National Monument,NE,MWR,AGFO,1.0


In [9]:
regionlookup.rename(columns={'Park Full Name':"park", "Reg":"region","Alpha":"alpha"}, inplace = True)
region_subset = regionlookup[['park', 'region', 'alpha', "State"]]
region_subset.head

<bound method NDFrame.head of                                                   park region alpha  \
0    Abraham Lincoln Birthplace National Historical...    SER  ABLI   
1                                 Acadia National Park    NER  ACAD   
2                       Adams National Historical Park    NER  ADAM   
3              African Burial Ground National Monument    NER  AFBG   
4                  Agate Fossil Beds National Monument    MWR  AGFO   
5                                   Alagnak Wild River    AKR  ALAG   
6            Alibates Flint Quarries National Monument    IMR  ALFL   
7                  Ala Kahakai National Historic Trail     PW  ALKA   
8    Allegheny Portage Railroad National Historic Site    NER  ALPO   
9                     Amistad National Recreation Area    IMR  AMIS   
10                              American Memorial Park     PW  AMME   
11                Andersonville National Historic Site    SER  ANDE   
12            Aniakchak National Monument and P

In [10]:
merged_df = pd.merge(df_update, region_subset, on="park", how="inner")
merged_df.head

<bound method NDFrame.head of           date                                        park  \
0   2024-03-28                   Crater Lake National Park   
1   2024-03-28                   Crater Lake National Park   
2   2024-03-28                   Crater Lake National Park   
3   2024-03-28                   Crater Lake National Park   
4   2024-03-28                   Crater Lake National Park   
5   2024-03-28                   Crater Lake National Park   
6   2024-03-28                   Crater Lake National Park   
7   2024-03-28                   Crater Lake National Park   
8   2024-03-28                   Crater Lake National Park   
9   2024-03-28                   Crater Lake National Park   
10  2024-03-28                   Crater Lake National Park   
11  2024-03-28                   Crater Lake National Park   
12  2024-03-28                   Crater Lake National Park   
13  2024-03-28                   Crater Lake National Park   
14  2024-03-28                    Shenan

In [11]:
df_old_update = pd.read_csv("./TDx_tracker.csv")

In [12]:
df_update = pd.concat([merged_df, df_old_update])
df_update.head

<bound method NDFrame.head of           date                                        park  \
0   2024-03-28                   Crater Lake National Park   
1   2024-03-28                   Crater Lake National Park   
2   2024-03-28                   Crater Lake National Park   
3   2024-03-28                   Crater Lake National Park   
4   2024-03-28                   Crater Lake National Park   
5   2024-03-28                   Crater Lake National Park   
6   2024-03-28                   Crater Lake National Park   
7   2024-03-28                   Crater Lake National Park   
8   2024-03-28                   Crater Lake National Park   
9   2024-03-28                   Crater Lake National Park   
10  2024-03-28                   Crater Lake National Park   
11  2024-03-28                   Crater Lake National Park   
12  2024-03-28                   Crater Lake National Park   
13  2024-03-28                   Crater Lake National Park   
14  2024-03-28                    Shenan

In [13]:
null_region_count = merged_df['region'].isnull().sum()
null_region_count

0

In [14]:
if null_region_count == 0:
    merged_df.to_csv("./TDx_tracker.csv", index=False)
else:
    null_region_rows = joined[joined['region'].isnull()]
    print("Rows with a null Region column:")
    print(null_region_rows)

In [15]:
path = "Park Unit Scraping Information.csv"
parks = pd.read_csv(path, encoding='latin-1')
parks.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Abraham Lincoln Birthplace,Abraham Lincoln Birthplace National Historical...,National Historical Park,KY,SER,ABLI,1.0
1,Acadia,Acadia National Park,National Park,ME,NER,ACAD,1.0
2,Adams,Adams National Historical Park,National Historical Park,MA,NER,ADAM,1.0
3,African Burial Ground,African Burial Ground National Monument,National Monument,NY,NER,AFBG,1.0
4,Agate Fossil Beds,Agate Fossil Beds National Monument,National Monument,NE,MWR,AGFO,1.0


In [16]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/alerts?limit=1000&api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [17]:
id_list = []
url_list = []
title_list = []
parkCode_list = []
description_list = []
category_list = []
relatedRoadEvents_list = []
lastIndexedDate_list = []

for x in df['data']:
    #print(x)
    id_list.append(x['id'])
    url_list.append(x['url']) 
    title_list.append(x['title']) 
    parkCode_list.append(x['parkCode'].upper()) 
    description_list.append(x['description'])
    category_list.append(x['category']) 
    relatedRoadEvents_list.append(x['relatedRoadEvents'])
    lastIndexedDate_list.append(x['lastIndexedDate'])


In [18]:
alerts_df = pd.DataFrame({
    'id': id_list, 
    'url': url_list, 
    'title': title_list,
    'park alpha': parkCode_list,
    'description': description_list,
    'category': category_list,
    'relatedRoadEvents':relatedRoadEvents_list,
    'LastIndexedDate':lastIndexedDate_list
})

In [19]:
alerts_df2 = alerts_df.merge(parks,left_on='park alpha', right_on="Alpha", how = "left")
alerts_df2.head()

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,156A62E5-EE42-49F6-9C8E-13A422C69B57,https://www.nps.gov/moja/road-conditions.htm,Mojave Road CLOSED across Soda Lake Crossing,MOJA,March rains have caused the Soda Lake area to ...,Park Closure,[{'title': 'Soda Lake Crossing CLOSED again fo...,2024-03-27 23:29:20.0,Mojave,Mojave National Preserve,National Preserve,CA,PWR,MOJA,1.0
1,20B0C333-AEBC-4985-AE34-A3814EB3DC2F,https://forecast.weather.gov/MapClick.php?lon=...,Road from Park Headquarters to Rim Village is ...,CRLA,The road is temporarily closed due to the late...,Park Closure,[{'title': 'Road from Park Headquarters to Rim...,2024-03-27 19:50:04.0,Crater Lake,Crater Lake National Park,National Park,OR,PWR,CRLA,1.0
2,BC3D21B6-61CA-4538-8F8C-4CB22C36697E,,Temporary Vehicle Restriction on Mauna Loa Road,HAVO,"Mauna Loa Road will be restricted to hiking, b...",Park Closure,[],2024-03-27 18:39:47.0,Hawaii Volcanoes,Hawai'i Volcanoes National Park,National Park,HI,PWR,HAVO,1.0
3,DBDF0CB7-B6F5-4F31-94D7-F9686A20CF26,,Casa Grande Ruins Picnic Area is closed throug...,CAGR,Casa Grande Ruins picnic area is closed March ...,Information,[],2024-03-27 17:04:43.0,Casa Grande Ruins,Casa Grande Ruins National Monument,National Monument,AZ,IMR,CAGR,1.0
4,0398152F-D6D3-4ED2-8D2B-686194F5B6E5,,Road Construction Delays April 1 to Mid-August...,BICA,Road construction in the park from April 1 to ...,Information,[],2024-03-27 16:56:49.0,Bighorn Canyon,Bighorn Canyon National Recreation Area,National Recreation Area,"MT,WY",IMR,BICA,1.0


In [20]:
alerts_df2.to_csv("alerts_api.csv", index=False)

In [21]:
has_alerts = set(alerts_df2['park alpha'])
has_tdx = set(merged_df['alpha'])
has_both = has_alerts.intersection(has_tdx)
only_alerts = has_alerts - has_tdx
only_tdx = has_tdx - has_alerts
status = pd.DataFrame(columns=['Park Alpha', 'Status'])
status['Park Alpha'] = list(has_both) + list(only_alerts) + list(only_tdx)
status['Status'] = (['Appears in both'] * len(has_both)) + (['Alerts Only'] * len(only_alerts)) + (['TDx Only'] * len(only_tdx))
status

,Park Alpha,Status
0,MOJA,Appears in both
1,DEVA,Appears in both
2,CRLA,Appears in both
3,BRCA,Appears in both
4,PORE,Appears in both
5,BLCA,Appears in both
6,GUMO,Appears in both
7,ORPI,Appears in both
8,YOSE,Appears in both
9,SHEN,Appears in both


In [22]:
status.to_csv("combined_status.csv", index=False)